In [1]:
import torch
import torchvision
from torch import nn
import torchvision.transforms as transforms
from VGG_Face_torch import VGG_Face_torch
import argparse
import torch.optim as optim
from torch.autograd import Variable
import os
import csv
import cv2
import matplotlib.pyplot as plt
from skimage import io, transform
from PIL import Image

In [2]:
# parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
batch_size = 16
test_batch_size =16
epochs = 100
lr = 0.001
momentum = 0.9
seed=1
log_interval = 30
kwargs = {'num_workers': 4, 'pin_memory': True}


In [3]:
EXP_TYPE = 'Classification/'
textMAIN = '/timo/datasets/Dua/GRU-test/Data-Combinations/'
textPATH = textMAIN+EXP_TYPE
COMB = ['BDI','AVEC','AVEC-BDI','AVEC-TEST','BDI-TEST']
NUM = ['1','2','3','4','5']
trainlist=textPATH+COMB[0]+'train'+NUM[0]+'.txt'
testlist=textPATH+COMB[0]+'test'+NUM[0]+'.txt'


In [4]:
# https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
class DepressionDataset():
    """Face Landmarks dataset."""

    def __init__(self, csv_file, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        with open(csv_file, 'r') as file:
            self.lines = file.readlines()
        self.transform = transform

    def __len__(self):
        return len(self.lines)

    def __getitem__(self, idx):
        idx = len(self.lines)

        for i in range(idx):
            info = self.lines[i].rstrip()
            par_name = info.split(" ")[0]
            label = int(info.split(" ")[1])
            imgs = os.listdir(par_name)
            imgs.sort()
            for img in range(idx):
                img_path = par_name+"/"+imgs[img]
                image = Image.fromarray(io.imread(img_path))
#                 print(image)
                sample = {'image': image, 'label': label}

        if self.transform:
            transform_img = self.transform(sample['image'])
#             print(transform_img)
            sample = {'image': transform_img, 'label': sample['label']}
    
        return sample

In [5]:
transform_train = transforms.Compose([transforms.RandomHorizontalFlip(),
#                                  transforms.ToPILImage(),
                                 transforms.Resize(256),
                                #  transforms.RandomResizedCrop((224,224), scale=(0.875, 1.125), ratio=(1.0, 1.0)),
                                #  transforms.CenterCrop(224),
                                 transforms.RandomCrop(224),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.507395516207, ),(0.255128989415, ))
                                ])

transform_test  = transforms.Compose([transforms.Resize(224),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.507395516207, ),(0.255128989415, ))
                                ])


# train_data = torchvision.datasets.ImageFolder('./train',transform=transform_train)
# test_data = torchvision.datasets.ImageFolder('./test',transform=transform_test)
train_data = DepressionDataset(csv_file = trainlist,transform=transform_train)
test_data = DepressionDataset(csv_file = testlist,transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_data,   batch_size=test_batch_size, shuffle=False, **kwargs)

In [6]:
class average_meter(object):

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [7]:
class VGG_Net(nn.Module):
    def __init__(self, model):
        super(VGG_Net, self).__init__()

        self.pre_model = nn.Sequential(*list(model.children())[:-1])
        # self.dropout = nn.Dropout(p=0.8)
        self.classifier = nn.Linear(4096, 2)

    def forward(self, x):
        x = self.pre_model(x)
        # x = self.dropout(x)
        x = self.classifier(x)

        return x 

In [8]:
model_emotion = VGG_Face_torch
model_emotion.load_state_dict(torch.load('VGG_Face_torch.pth'))
model = VGG_Net(model_emotion).cuda()


loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum ,weight_decay= 0.0005,nesterov=True)
scheduler = optim.lr_scheduler.StepLR(optimizer,step_size=20, gamma=0.1)


In [9]:
def train(epoch):
    
    losses = average_meter()
    accuracy = average_meter()
    model.train()

    for batch_idx,sample in enumerate(train_loader):
#         print(data)
        data, target = Variable(sample['image']).cuda(), Variable(sample['label']).cuda()
        output = model(data)
        print(output.size())
        loss = loss_function(output, target)
#         print('output:',output.max(1))
        losses.update(loss.data, data.size(0))
#         print('max data:',output.data[1])
        pred = output.data.max(1)[1]
        prec = pred.eq(target.data).cpu().sum()
        accuracy.update(float(prec) / data.size(0), data.size(0))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

#         if batch_idx % log_interval == 0:
#             print('Train Epoch: {}\t'
#                   'Batch: [{:5d}/{:5d} ({:3.0f}%)]\t'                     
#                   'Loss: {:.6f}'.format(
#                       epoch, batch_idx * len(data), len(train_data),
#                       100. * batch_idx / len(train_loader), losses.val))
#             print('Training accuracy:', accuracy.val )

In [10]:
def val():
    losses = average_meter()
    accuracy = average_meter()

    model.eval()

    for sample in test_loader:

        data, target = Variable(sample['image'],volatile=True).cuda(), Variable(sample['label'],volatile=True).cuda()
        output = model(data)
        loss = loss_function(output, target)
        losses.update(loss.data, data.size(0))

        pred = output.data.max(1)[1]
        prec = pred.eq(target.data).cpu().sum()
        accuracy.update(float(prec) / data.size(0), data.size(0))

#     print('\nTest: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
#         losses.avg, int(accuracy.sum), len(test_data), 100. * accuracy.avg))

    return accuracy.avg

In [ ]:
def main():

    best_model = model
    best_accuray = 0.0

    for epoch in range(1, epochs + 1):

        train(epoch)
        val_accuracy = val()
        scheduler.step()


        if best_accuray < val_accuracy:
            best_model   = model
            best_accuray = val_accuracy


    print ("The best model has an accuracy of " + str(best_accuray))
    torch.save(best_model.state_dict(), 'best.model')


if __name__ == '__main__':
    main()

torch.Size([16, 2])
torch.Size([16, 2])
torch.Size([4, 2])


/home/duaa/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  if __name__ == '__main__':


torch.Size([16, 2])
torch.Size([16, 2])
torch.Size([4, 2])
torch.Size([16, 2])
torch.Size([16, 2])
torch.Size([4, 2])


In [ ]:
number = [0.3206, 0.3167, 0.2984, 0.2252]
max(number)